In [1]:
import pandas as pd
import numpy as np
import json
import requests
import lxml.html as lh
import re
from bs4 import BeautifulSoup

In [28]:
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'

# Wayback Machine work!!!
#url = 'https://web.archive.org/web/20201130212042/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
#url = 'https://web.archive.org/web/20200720171507/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
# TODO - Wayback machine adds two extra tables at top we need to skip past
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [29]:
tables = soup.find_all('table')
#[len(a) for a in a[0].find_all('tr')]

In [30]:
tmp = re.split('/| ', tables[0].find_all('caption')[0].get_text())
reporting_date = '-'.join(tmp[0:3])
print(reporting_date)

11-23-2020


In [31]:
reporting_date

'11-23-2020'

In [32]:
# Create list of facilities
rows = tables[2].find_all('tr')
# Get List of fields from Header Row
fieldList = []
fields = rows[0].find_all('th')
for field in fields:
    fieldList.append(field.get_text())
#assert fieldList == ['FACILITY NAME', 'COUNTY','TYPE', 'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS', 'TOTAL RESIDENT DEATHS',
#                     'NEW STAFF CASES', 'TOTAL STAFF CASES', 'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS']
num_facilities = 0
facilities = []
first_row = True
for row in rows:
    if first_row:
        first_row = False
        continue
    facility = {}
    tds = row.find_all('td')
    num_reported_vals = 0
    facility['ReportingDate'] = reporting_date
    for i in range(len(tds)):
        val = tds[i].get_text()
        if val == '--':
            val = float('Nan')
        else:
            num_reported_vals = num_reported_vals +1
        facility[fieldList[i]] = val   
    facilities.append(facility)
print(len(facilities))

1352


In [33]:
facilities_df = pd.read_json(json.dumps(facilities), dtype={'ReportingDate': np.str, 'FACILITY NAME': str, 'COUNTY': str, 'TYPE': str,
                                      'NEW RESIDENT CASES': float, 'TOTAL RESIDENT CASES': float, 'NEW RESIDENT DEATHS': float,
                      'TOTAL RESIDENT DEATHS': float, 'NEW STAFF CASES': float, 'TOTAL STAFF CASES': float,
                      'NEW STAFF DEATHS': float, 'TOTAL STAFF DEATHS': float})

In [34]:
facilities_df.to_csv('Reporting_data/MI_' + reporting_date + 'LTC_data.csv', index=False)

In [35]:
facilities_df.columns

Index(['ReportingDate', 'CCN/LICENSE', 'FACILITY NAME', 'COUNTY', 'TYPE',
       'COMPLIANCE', 'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES',
       'NEW RESIDENT DEATHS', 'TOTAL RESIDENT DEATHS', 'NEW STAFF CASES',
       'TOTAL STAFF CASES', 'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS'],
      dtype='object')

In [36]:
cols = ['NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS',
       'TOTAL RESIDENT DEATHS', 'NEW STAFF CASES', 'TOTAL STAFF CASES',
       'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS']
print(facilities_df['TYPE'].value_counts())
print(facilities_df[cols].sum().astype(int))

AFC    607
SNF    440
HFA    305
Name: TYPE, dtype: int64
NEW RESIDENT CASES        1038
TOTAL RESIDENT CASES     13332
NEW RESIDENT DEATHS        146
TOTAL RESIDENT DEATHS     3123
NEW STAFF CASES           1294
TOTAL STAFF CASES         9520
NEW STAFF DEATHS             0
TOTAL STAFF DEATHS          43
dtype: int64


In [40]:
facilities_df.sort_values(by='TYPE', ascending=False).head(30)

,ReportingDate,CCN/LICENSE,FACILITY NAME,COUNTY,TYPE,COMPLIANCE,NEW RESIDENT CASES,TOTAL RESIDENT CASES,NEW RESIDENT DEATHS,TOTAL RESIDENT DEATHS,NEW STAFF CASES,TOTAL STAFF CASES,NEW STAFF DEATHS,TOTAL STAFF DEATHS
0,11-23-2020,235628,Jamieson Nursing Home,ALCONA,SNF,,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
736,11-23-2020,235003,Manistee County Medical Care Facility,MANISTEE,SNF,,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0
714,11-23-2020,235506,Shelby Nursing Center,MACOMB,SNF,,3.0,88.0,0.0,13.0,2.0,31.0,0.0,0.0
715,11-23-2020,235443,Shorepoint Nursing Center,MACOMB,SNF,,0.0,63.0,0.0,20.0,6.0,38.0,0.0,1.0
716,11-23-2020,235408,St. Anthony Nursing Healthcare,MACOMB,SNF,,0.0,87.0,0.0,19.0,0.0,18.0,0.0,0.0
720,11-23-2020,38,The Villa At City Center,MACOMB,SNF,,0.0,59.0,0.0,17.0,0.0,27.0,0.0,0.0
721,11-23-2020,235528,The Village of East Harbor,MACOMB,SNF,,0.0,39.0,0.0,7.0,1.0,20.0,0.0,0.0
730,11-23-2020,235259,Warren Woods Health and Rehab Center,MACOMB,SNF,,0.0,49.0,0.0,24.0,0.0,21.0,0.0,1.0
731,11-23-2020,235704,WellBridge of Romeo,MACOMB,SNF,,0.0,2.0,0.0,2.0,1.0,12.0,0.0,0.0
733,11-23-2020,235714,Windemere Park Health and Rehab Center,MACOMB,SNF,,3.0,21.0,0.0,1.0,0.0,17.0,0.0,0.0
